In [12]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd

In [80]:
date = '20190907'
url = 'https://www.playsport.cc/predictgame.php?action=scale&allianceid=2&gametime={}&sid=0'.format(date)
page = urllib.request.urlopen(url)

soup = BeautifulSoup(page)

In [176]:
def findClass(data, element, _class=None):
    if _class:
        return data.find_all(element, attrs={'class': _class})
    return data.find_all(element)

def get_scores(text):
    for game in text.find_all('td', attrs={'class' : 'scores'}):
        host_score = int(game.find_all('li')[0].text.strip())
        guest_score = int(game.find_all('li')[2].text.strip())
        yield host_score, guest_score
        
def get_rates(soup):
    for game in findClass(soup, 'tr', 'game-set'):
        for data in findClass(game, 'td', re.compile('td-bank-bet\d')):
            for rate in findClass(data, 'span', re.compile('data-wrap')):
                yield findClass(rate, 'strong')[0].string, findClass(rate, 'span')[0].string.replace(', ', '').strip()
                
                

In [262]:
class Game:
    def __init__(self, soup_even, soup_odd):
        self._home_score, self._guest_score = self.GetScores(soup_even)
        self._home_line, self._guest_line = self.GetBettingLine(soup_even), self.GetBettingLine(soup_odd)

    def GetScores(self, soup):
        ho_score = int(soup.find_all('li')[0].text.strip())
        guest_score = int(soup.find_all('li')[2].text.strip())
        return home_score, guest_score
    
    def GetBettingLine(self, soup):
        odds_str = soup.find_all('td', attrs={'class' : re.compile('td-bank-bet03')})[0]
        return float(odd_str.find_all('span')[1].text)

In [263]:
game_set = soup.find_all('tr', attrs={'class' : 'game-set'})
it = iter(game_set)
slot = [Game(x, next(it)) for x in it]

In [268]:
slot[0]._guest_line

1.8